<h1 style="text-align:center">
    Pump it Up: Data Mining the Water Table
</h1>

<h2 style="text-align:center">
    Nombre y apellidos 
    <br/> <br/>
    Fecha
</h2>

In [133]:
import pickle
import pandas as pd 
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Cargado datos

In [134]:
# Variable objetivo 
target = pd.read_csv("train_target.csv")

# Remplazo valores variable objetivo 
target = target.replace({'status_group': {'functional' : 1, 
                                          'non functional' : 0, 
                                          'functional needs repair' : 2}})

# Variables independientes 
values = pd.read_csv("train_values.csv")

test = pd.read_csv("test_values.csv")

# Unión variables independientes y variable dependiente  
data = values.merge(target, on = 'id')
data.set_index(['id'], inplace=True)

ID = test['id']
test.set_index(['id'], inplace=True)

In [135]:
data['funder'] = data['funder'].fillna('Missing')
test['funder'] = test['funder'].fillna('Missing')

In [136]:
chars = []
for fund in data.funder:
    if fund not in chars:
        chars.append(fund)

for fun in test.funder:
        if fun not in chars:
            chars.append(fun)

In [137]:
equi = {}
num = 1 
for char in chars: 
    equi[char] = num
    num += 1

In [138]:
funders = []
for fund in data.funder:
    word = str(equi[fund])
    funders.append(word)

In [139]:
fundors = []
for fund in test.funder:
    word = str(equi[fund])
    fundors.append(word)

In [141]:
data['funder'] = pd.Series([int(num) for num in funders],index = data.index).astype('int')

In [142]:
test['funder'] = pd.Series([int(num) for num in fundors],index = test.index).astype('int')

# EDA

In [4]:
data.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,status_group
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475,0.688434
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547,0.599877
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,1.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,1.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,2.000000


In [3]:
test.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14850.000000,14850.000000,14850.000000,1.485000e+04,14850.000000,14850.000000,14850.000000,14850.000000,14850.000000
mean,322.826983,655.147609,34.061605,-5.684724e+00,0.415084,15.139057,5.626397,184.114209,1289.708350
std,2510.968644,691.261185,6.593034,2.940803e+00,8.167910,17.191329,9.673842,469.499332,955.241087
min,0.000000,-57.000000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.069455,-8.443970e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,344.000000,34.901215,-5.049750e+00,0.000000,12.000000,3.000000,20.000000,1986.000000
75%,25.000000,1308.000000,37.196594,-3.320594e+00,0.000000,17.000000,5.000000,220.000000,2004.000000
max,200000.000000,2777.000000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [48]:
test.isna().sum()

amount_tsh                  0
date_recorded               0
funder                    869
gps_height                  0
installer                 877
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 99
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            821
recorded_by                 0
scheme_management         969
scheme_name              7092
permit                    737
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity                    0
quantity_g

In [ ]:
data.dtypes

In [167]:
data.recorded_by.value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [ ]:
for code in data.extraction_type_group.unique():
    print(code + ': ' + str(data[data['extraction_type_group'] == code]['extraction_type'].unique()))

In [ ]:
for code in data.extraction_type_class.unique():
    print(code + ': ' + str(data[data['extraction_type_class'] == code]['extraction_type_group'].unique()))

In [ ]:
for code in data.management_group.unique():
    print(code + ': ' + str(data[data['management_group'] == code]['management'].unique()))

In [ ]:
for code in data.payment_type.unique():
    print(code + ': ' + str(data[data['payment_type'] == code]['payment'].unique()))

In [ ]:
for code in data.quality_group.unique():
    print(code + ': ' + str(data[data['quality_group'] == code]['water_quality'].unique()))

In [ ]:
for code in data.quantity_group.unique():
    print(code + ': ' + str(data[data['quantity_group'] == code]['quantity'].unique()))

In [ ]:
for code in data.source_type.unique():
    print(code + ': ' + str(data[data['source_type'] == code]['source'].unique()))

In [ ]:
for code in data.source_class.unique():
    print(code + ': ' + str(data[data['source_class'] == code]['source_type'].unique()))

In [ ]:
for code in data.waterpoint_type_group.unique():
    print(code + ': ' + str(data[data['waterpoint_type_group'] == code]['waterpoint_type'].unique()))

# Transformación variables 

In [143]:
data['y_recorded'] = [int(anno.split('-')[0]) for anno in data['date_recorded']]
data['m_recorded'] = [int(mes.split('-')[1]) for mes in data['date_recorded']]
data['d_recorded'] = [int(dia.split('-')[2]) for dia in data['date_recorded']]

test['y_recorded'] = [int(anno.split('-')[0]) for anno in test['date_recorded']]
test['m_recorded'] = [int(mes.split('-')[1]) for mes in test['date_recorded']]
test['d_recorded'] = [int(dia.split('-')[2]) for dia in test['date_recorded']]

y = data['status_group']

to_drop = ['date_recorded', 'recorded_by', 'extraction_type_group', 'extraction_type_class', 'management_group', 'payment_type',
           'quality_group','quantity_group', 'source_type', 'source_class', 'waterpoint_type_group','status_group']

to_drop = ['installer', 'wpt_name', 'scheme_management',
           'subvillage', 'lga', 'ward', 'basin',
           'scheme_name', 'payment', 'management',
           'waterpoint_type_group'] + to_drop

data.drop(columns = to_drop, inplace =True)
test.drop(columns = to_drop[:-1], inplace =True)

In [88]:
data.columns

Index(['amount_tsh', 'funder', 'gps_height', 'longitude', 'latitude',
       'num_private', 'region', 'region_code', 'district_code', 'population',
       'public_meeting', 'permit', 'construction_year', 'extraction_type',
       'water_quality', 'quantity', 'source', 'waterpoint_type', 'y_recorded',
       'm_recorded', 'd_recorded'],
      dtype='object')

In [144]:
for i in data.select_dtypes(include=['object']):
    print(i, len(data[i].unique()))

region 21
public_meeting 3
permit 3
extraction_type 18
water_quality 8
quantity 5
source 10
waterpoint_type 7


In [145]:
for i in test.select_dtypes(include=['object']):
    print(i, len(test[i].unique()))

region 21
public_meeting 3
permit 3
extraction_type 17
water_quality 8
quantity 5
source 10
waterpoint_type 7


In [146]:
# One Hot Encoding with get_dummy variables and drop first
data = pd.get_dummies(data)
test = pd.get_dummies(test)

In [147]:
data.drop(columns = 'extraction_type_other - mkulima/shinyanga', inplace =True)

# Partición Training - Test

In [148]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=1/5, random_state=42)

# Evaluación Modelos

## a) Random Forest

In [149]:
forest_clf = RandomForestClassifier(bootstrap =  True, criterion = 'gini', n_estimators = 1000)
forest_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [150]:
forest_training_preds = forest_clf.predict(x_train)
forest_training_accuracy = accuracy_score(y_train, forest_training_preds)
forest_training_accuracy

0.9982744107744108

In [151]:
forest_val_preds = forest_clf.predict(x_test) 
forest_val_accuracy = accuracy_score(y_test, forest_val_preds)
forest_val_accuracy

0.8077441077441078

In [152]:
importance = forest_clf.feature_importances_
for num in range(len(importance)):
    print(data.columns[num] + ': ' +  str(importance[num]))

amount_tsh: 0.02914576229414626
funder: 0.048186643624777856
gps_height: 0.0647170193073375
longitude: 0.1299493096978755
latitude: 0.12829681067444298
num_private: 0.0015556392567934467
region_code: 0.0167812252232714
district_code: 0.025510707649708322
population: 0.0448523967494227
construction_year: 0.049531485447835644
y_recorded: 0.006933362561784466
m_recorded: 0.02218280163358724
d_recorded: 0.06384904706599834
region_Arusha: 0.0018248857705764898
region_Dar es Salaam: 0.00038429425947746846
region_Dodoma: 0.0012567805645867791
region_Iringa: 0.004786984832667543
region_Kagera: 0.0015338301180923624
region_Kigoma: 0.002705036022669622
region_Kilimanjaro: 0.002279951184319935
region_Lindi: 0.0010807878362289226
region_Manyara: 0.0011255766808890432
region_Mara: 0.0016344907253377863
region_Mbeya: 0.001559192049051213
region_Morogoro: 0.001995823300035679
region_Mtwara: 0.0011731437130539188
region_Mwanza: 0.0017771384810950645
region_Pwani: 0.001471483728765967
region_Rukwa: 0.0

## b) XGBoost

In [153]:
for name in x_train.columns:
    if '[' in name:
        x_train = x_train.rename(columns = {name : name.replace('[','').replace(']','') })

for name in x_test.columns:
    if '[' in name:
        x_test = x_test.rename(columns = {name : name.replace('[','').replace(']','') })       

In [154]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False)
xgb_clf.fit(x_train, y_train)

[12:04:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [155]:
xgb_training_preds = xgb_clf.predict(x_train)
xgb_training_accuracy = accuracy_score(y_train, xgb_training_preds)
xgb_training_accuracy

0.8398569023569024

In [156]:
xgb_val_preds = xgb_clf.predict(x_test)
xgb_val_accuracy = accuracy_score(y_test, xgb_val_preds)
xgb_val_accuracy

0.795959595959596

## Grid

In [157]:
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [158]:
model_grid = GridSearchCV(estimator=RandomForestClassifier(),
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)

In [159]:
model_grid.fit(data, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'n_estimators': [100, 300, 500, 800, 1000]},
             scoring='accuracy')

In [160]:
print(model_grid.best_params_)

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 800}


In [161]:
print(model_grid.best_score_)

0.8078282828282829


# Entrenamiento Modelo

In [162]:
modelo_final = RandomForestClassifier(bootstrap =  True, criterion = 'entropy', n_estimators = 800)
modelo_final.fit(data, y)

RandomForestClassifier(criterion='entropy', n_estimators=800)

In [163]:
forest_training_preds = modelo_final.predict(data)
forest_training_accuracy = accuracy_score(y, forest_training_preds)
forest_training_accuracy

0.9980639730639731

# Prediciones

In [164]:
final_preds = modelo_final.predict(test)

In [165]:
preds = pd.concat([ID, pd.Series(final_preds, name = 'status_group')], axis=1)

# Replace target values - there are three classes
preds = preds.replace({'status_group': {1 : 'functional', 
                                        0 : 'non functional', 
                                        2 : 'functional needs repair'}})
preds

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [166]:
preds.to_csv('preds.csv', index = False)  

In [29]:
pkl_filename = "Forest_812.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(modelo_final, file)

## Cargado modelo

In [32]:
with open('Forest_812.pkl', 'rb') as file:
    Forest_80 = pickle.load(file)

In [33]:
forest_training_preds = Forest_80.predict(test)

In [34]:
preds1 = pd.concat([ID, pd.Series(forest_training_preds, name = 'status_group')], axis=1)

# Replace target values - there are three classes
preds1 = preds1.replace({'status_group': {1 : 'functional', 
                                        0 : 'non functional', 
                                        2 : 'functional needs repair'}})
preds1

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional
